In [ ]:
import pandas as pd
import numpy as np

In [ ]:
weather_data = pd.read_csv('weather_features.csv')

In [ ]:
# only use data from Madrid
weather_data = weather_data[weather_data['city_name']=='Madrid']

In [ ]:
weather_data.index = pd.to_datetime(weather_data['dt_iso'])

In [ ]:
#drop columns that are not needed, rain , snow, clouds, wind direction are not needed
weather_data.drop(columns=['dt_iso', 'city_name', 'rain_1h', 'rain_3h', 'snow_3h', 'clouds_all', 'wind_deg'], inplace=True)

In [ ]:
# for categorical columns show value counts
for col in weather_data.columns:
    if weather_data[col].dtype == 'object':
        print(col, weather_data[col].nunique())

In [ ]:
# since weather_description and weather_icon have a lot of unique values, drop them
weather_data.drop(columns=['weather_icon', 'weather_description', 'weather_id'], inplace=True)

In [ ]:
energy_data = pd.read_csv('energy_dataset.csv')

In [ ]:
energy_data.index = pd.to_datetime(energy_data['time'])
# drop columns that are not needed
energy_data.drop(columns=['time', "generation hydro pumped storage aggregated", "forecast wind offshore eday ahead"], inplace=True)

In [ ]:
energy_data.dropna(inplace=True)

In [ ]:
# sum all the generation types to get the total generation
energy_data['total_generation'] = energy_data[energy_data.columns[:20]].sum(axis=1)

In [ ]:
# drop all the generation types
energy_data.drop(columns=energy_data.columns[:20], inplace=True)

In [ ]:
# drop all the forecast columns
forecast_columns = [col for col in energy_data.columns if 'forecast' in col]
energy_data.drop(columns=forecast_columns, inplace=True)
energy_data.drop(columns=['price day ahead'], inplace=True)

In [ ]:
merged_data = pd.merge(energy_data, weather_data, how='inner', left_index=True, right_index=True)
merged_data.index = pd.to_datetime(merged_data.index, utc=True)

In [ ]:
columns_scale_std = ["total load actual", "price actual", "total_generation", 'temp', 'temp_min', 'temp_max']
columns_scale_min_max = ['pressure', 'humidity', 'wind_speed']

In [ ]:
merged_data[columns_scale_std] = (merged_data[columns_scale_std] - merged_data[columns_scale_std].mean()) / merged_data[columns_scale_std].std()

In [ ]:
merged_data[columns_scale_min_max] = (merged_data[columns_scale_min_max] - merged_data[columns_scale_min_max].min()) / (merged_data[columns_scale_min_max].max() - merged_data[columns_scale_min_max].min())

In [ ]:
# get dummy variables for weather_main
merged_data = pd.get_dummies(merged_data, columns=['weather_main'])

In [ ]:
# add time features
merged_data['hour'] = merged_data.index.hour
merged_data['day_of_week'] = merged_data.index.dayofweek
merged_data['day_of_month'] = merged_data.index.day
merged_data['month'] = merged_data.index.month
merged_data['day_of_year'] = merged_data.index.year

In [ ]:
# get time encodings, as sine and cosine
merged_data['hour_sin'] = np.sin(merged_data['hour']*(2.*np.pi/24))
merged_data['hour_cos'] = np.cos(merged_data['hour']*(2.*np.pi/24))
merged_data['day_of_week_sin'] = np.sin(merged_data['day_of_week']*(2.*np.pi/7))
merged_data['day_of_week_cos'] = np.cos(merged_data['day_of_week']*(2.*np.pi/7))
merged_data['day_of_month_sin'] = np.sin(merged_data['day_of_month']*(2.*np.pi/30))
merged_data['day_of_month_cos'] = np.cos(merged_data['day_of_month']*(2.*np.pi/30))
merged_data['month_sin'] = np.sin((merged_data['month']-1)*(2.*np.pi/12))
merged_data['month_cos'] = np.cos((merged_data['month']-1)*(2.*np.pi/12))
merged_data['day_of_year_sin'] = np.sin((merged_data['day_of_year']-1)*(2.*np.pi/365))
merged_data['day_of_year_cos'] = np.cos((merged_data['day_of_year']-1)*(2.*np.pi/365))

In [ ]:
merged_data.drop(columns=['hour', 'day_of_week', 'day_of_month', 'month', 'day_of_year'], inplace=True)

In [ ]:
merged_data.to_csv('merged_data.csv', index_label='time')

In [ ]:
# example use
merged_data = pd.read_csv('merged_data.csv', index_col='time')